In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)  # for reproducibility

2.4.1


In [18]:
# 데이터 불러오기
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

nb_classes = 7  # 0 ~ 6

# Y값을 원핫 값으로 변경 
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [21]:
# 소프트맥스 분류기 구현

W = tf.Variable(tf.random.normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b]

def logit_fn(X):
    return tf.matmul(X,W)+b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X,Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, from_logits=True)
    cost = tf.reduce_mean(cost_i)
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X,Y)
        grads = tape.gradient(loss, variables)
        return grads

def prediction(X,Y):
    pred = tf.argmax(hypothesis(X),1)
    # Y와 pred 일치하는지 계산
    correct_prediction = tf.equal(pred, tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [22]:
# 소프트맥스 분류기 학습하는 함수
def fit(X, Y, epochs=1000, verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads = grad_fn(X,Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            acc = prediction(X,Y).numpy()
            loss = cost_fn(X,Y).numpy()
            print('Steps: {}, Loss: {}, Acc: {}'.format(i+1, loss, acc))
            
fit(x_data, Y_one_hot)

Steps: 1, Loss: 4.067071437835693, Acc: 0.11881188303232193
Steps: 100, Loss: 0.7015846371650696, Acc: 0.7524752616882324
Steps: 200, Loss: 0.44547685980796814, Acc: 0.8613861203193665
Steps: 300, Loss: 0.3346084654331207, Acc: 0.9009901285171509
Steps: 400, Loss: 0.2687549889087677, Acc: 0.9207921028137207
Steps: 500, Loss: 0.22413121163845062, Acc: 0.9207921028137207
Steps: 600, Loss: 0.1916474848985672, Acc: 0.9504950642585754
Steps: 700, Loss: 0.16689886152744293, Acc: 0.9702970385551453
Steps: 800, Loss: 0.14744509756565094, Acc: 0.9702970385551453
Steps: 900, Loss: 0.13179805874824524, Acc: 0.9702970385551453
Steps: 1000, Loss: 0.11898202449083328, Acc: 0.9900990128517151
